In [11]:
import ast

source = open("test_packages/calculator-0.0.1/calculator/list_functions.py", "r").read()
tree = ast.parse(source)

In [12]:
dec = tree.body[2].body[4].decorator_list[0]
ast.get_source_segment(source, dec)

'staticmethod'